<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Enusre Long Running AWS RDS Instances have Reserved Instances purchased for them.</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-RDS-Instances-with-Low-CPU-Utilization"><u>Purchase Reserved Instances For Long Running AWS RDS Instances</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find Long Running AWS RDS Instances without Reserved Instances</a><br>2)<a href="#2" target="_self" rel="noopener"> Purchase Reserved Instance</a></p>

In [ ]:
if region == None:
    region = ''
threshold_int = int(threshold)
if reserved_instance_offering_id and not region:
    raise SystemExit("Provide a region for the Reserved Cache Instance Offering ID!")

<h3 id="Find-ECS-Clusters-with-Low-CPU-Utilization"><a id="1" target="_self" rel="nofollow"></a>Find Long Running AWS RDS Instances without Reserved Instances</h3>
<p>Using unSkript's Find Long Running AWS RDS Instances without Reserved Instances action, we will find RDS DB Instances that have been running for longer than a specified threshold and do not have reserved instances purchased for them.</p>
<blockquote>
<p>This action takes the following parameters: <code>region, threshold</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>clusters_without_reserved_instances</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime,timedelta, timezone
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions

from beartype import beartype
@beartype
def aws_get_long_running_rds_instances_without_reserved_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_get_long_running_rds_instances_without_reserved_instances(handle, region: str = "", threshold:int=10) -> Tuple:
    """aws_get_long_running_rds_instances_without_reserved_instances Gets all DB instances that are not m5 or t3.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: AWS Region.

        :type threshold: int
        :param threshold: Threshold(in days) to find long running RDS instances. Eg: 30, This will find all the instances that have been created a month ago.

        :rtype: A tuple with a Status,and list of DB instances that don't have reserved instances
    """
    result = []
    all_regions = [region]
    reservedInstancesPerRegion = {}
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            rdsClient = handle.client('rds', region_name=reg)
            response = rdsClient.describe_reserved_nodes()
            reservedInstancesPerType = {}
            if response['ReservedDBInstances']:
                for ins in response['ReservedDBInstances']:
                    reservedInstancesPerRegion[ins['DBInstanceClass']] = True
            else:
                continue
            reservedInstancesPerRegion[reg] = reservedInstancesPerType
        except Exception:
            pass
    for reg in all_regions:
        try:
            rdsClient = handle.client('rds', region_name=reg)
            response = aws_get_paginator(rdsClient, "describe_db_instances", "DBInstances")
            for instance in response:
                if instance['DBInstanceStatus'] == 'available':
                        uptime = datetime.now(timezone.utc) - instance['InstanceCreateTime']
                        if uptime > timedelta(days=threshold):
                            # Check if the cluster node type is present in the reservedInstancesPerRegion map.
                            reservedInstances = reservedInstancesPerRegion.get(reg)
                            if reservedInstances != None:
                                if reservedInstances.get(instance['DBInstanceClass']) == True:
                                    continue
                            db_instance_dict = {}
                            db_instance_dict["region"] = reg
                            db_instance_dict["instance_type"] = instance['DBInstanceClass']
                            db_instance_dict["instance"] = instance['DBInstanceIdentifier']
                            result.append(db_instance_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "threshold": "int(threshold_int)"
    }''')

task.configure(outputName="clusters_without_reserved_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_long_running_rds_instances_without_reserved_instances, lego_printer=aws_get_long_running_rds_instances_without_reserved_instances_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Clusters-without-reserved-instances">Create List of Clusters without reserved instances</h3>
<p>This action filters regions that have no clusters and creates a list of those that have them (without reserved instances).</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_clusters_without_reserved_instances</code></p>
</blockquote>

In [ ]:
all_clusters_without_reserved_instances = []
dummy = []
for res in clusters_without_reserved_instances:
    if type(res)==bool:
        if res == False:
            continue
    elif type(res)==list:
        if len(res)!=0:
            all_clusters_without_reserved_instances=res
print(all_clusters_without_reserved_instances)
task.configure(outputName="all_clusters_without_reserved_instances")

<h3 id="Delete-RDS-Instance"><a id="2" target="_self" rel="nofollow"></a>Purchase Reserved Instance</h3>
<p>This action Purchases Reserved Instances for the clusters found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>db_instance_count, region, reserved_instance_offering_id</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint


from beartype import beartype
@beartype
def aws_purchase_rds_reserved_instance_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_purchase_rds_reserved_instance(handle, region: str, reserved_instance_offering_id: str, db_instance_count:int=1) -> Dict:
    """aws_purchase_rds_reserved_instance returns dict of response.

        :type region: string
        :param region: AWS Region.

        :type reserved_instance_offering_id: string
        :param reserved_instance_offering_id: The unique identifier of the reserved instance offering you want to purchase.

        :type db_instance_count: int
        :param db_instance_count: The number of reserved instances that you want to purchase.

        :rtype: dict of response metatdata of purchasing a reserved instance
    """
    try:
        redshiftClient = handle.client('redshift', region_name=region)
        params = {
            'ReservedDBInstancesOfferingId': reserved_instance_offering_id,
            'DBInstanceCount': db_instance_count
            }
        response = redshiftClient.purchase_reserved_db_instances_offering(**params)
        return response
    except Exception as e:
        raise Exception(e)




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "reserved_instance_offering_id": "reserved_instance_offering_id",
    "db_instance_count": int(db_instance_count)
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_purchase_rds_reserved_instance, lego_printer=aws_purchase_rds_reserved_instance_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion</h3>
<p>In this Runbook, we were able to filter long running RDS instances without reserved nodes given a threshold number of days of creation and purchase instances for them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>